## Part 1
How many times is the `mul` instruction invoked?

- `set X Y` sets register `X` to the value of `Y`.
- `sub X Y` decreases register `X` by the value of `Y`.
- `mul X Y` sets register `X` to the result of multiplying the value contained in register `X` by the value of `Y`.
- `jnz X Y` jumps with an offset of the value of `Y`, but only if the value of `X` is not zero. (An offset of `2` skips the next instruction, an offset of `-1` jumps to the previous instruction.)

In [22]:
def parse(s):
    try:
        return int(s)
    except ValueError:
        return s

class Processor():
    def __init__(self, instructions):
        self.registers = {chr(ord('a') + i): 0 for i in range(8)}
        self.instructions, self.current_instruction, self.steps, self.mul_count = [], 0, 0, 0
        for ins_s in instructions:
            split = ins_s.split(' ')
            self.instructions.append((split[0], parse(split[1]), parse(split[2])))

    def __str__(self):
        r_print = '\n'.join([f'{register} = {value}' for register, value in self.registers.items()])
        i_print_l = []
        for i in range(len(self.instructions)):
            i_print_l.append(f'{i:02}{"*" if i == self.current_instruction else " "}{str(self.instructions[i])}')
        i_print = f'\nInstructions (current = {self.current_instruction:02}):\n' + '\n'.join(i_print_l)
        return f'Current step is {self.steps}, mul was called {self.mul_count} times\n\nRegisters:\n{r_print}\n{i_print}'

    def step(self, debug=False):
        cmd, x, y = self.instructions[self.current_instruction]
        if cmd == 'set':
            self.registers[x] = self.value_of(y)
        elif cmd == 'sub':
            self.registers[x] -= self.value_of(y)
        elif cmd == 'mul':
            self.registers[x] *= self.value_of(y)
            self.mul_count += 1
        elif cmd == 'jnz':
            if self.value_of(x) != 0:
                if debug:
                    i = self.instructions[self.current_instruction]
                    i = f'{i[0]} {i[1]} {i[2]}'.ljust(12)
                    print(f'{self.steps+1:02} #{self.current_instruction:02} {i} {self.registers}')
                self.current_instruction += self.value_of(y)
                self.steps += 1
                return
        else:
            raise ValueException(f'No such command: {cmd}')
        if debug:
            i = self.instructions[self.current_instruction]
            i = f'{i[0]} {i[1]} {i[2]}'.ljust(12)
            print(f'{self.steps+1:02} #{self.current_instruction:02} {i} {self.registers}')
        self.current_instruction += 1
        self.steps += 1

    def value_of(self, y):
        return y if isinstance(y, int) else self.registers[y]

    def run(self, max_steps=10000):
        for i in range(max_steps):
            if self.current_instruction > len(self.instructions) - 1:
                return
            self.step()
        raise Exception(f'Execution have not ended after {self.steps} steps.')

In [23]:
puzzle_in = [line[:-1] for line in open('in/day23.txt', 'r')]
proc = Processor(puzzle_in)
proc.run(100000)
print(f'Part 1 answer is: `mul` was called {proc.mul_count} times')
assert proc.mul_count == 6724

Part 1 answer is: `mul` was called 6724 times
